# EXAMEN DATASCIENCE II

Antes de nada realizar un git clone del proyecto de github. Para la realización del test, borrar las opciones incorrectas dejando sólamente la respuesta correcta

# ARIKETA 1 (2 puntos)

Elige la respuesta mas adecuada para las siguientes preguntas. Nota: Respuesta correcta +0.20. Respuesta incorrecta -0.10

### NOTA: Llegado a este punto, tienes que realizar un git commit y un git push. Esta parte del notebook no se puede modificar y no se puede empezar con el ejercicio 2 sin haber hecho los pasos previos. Si se hace un cambio en el test a lo largo del examen, no se corregirán los ejercicios restantes.

# ARIKETA 2 (2 puntos)

Dada la página web https://dreguera.github.io y utilizando la Distancia de Levenshtein, printear en pantalla cuantas veces aparece el término escrito correctamente

##### Librerias

In [1]:
from itertools import chain
import re
import string
import unidecode
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize

from lxml import html
import requests
import codecs

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import Levenshtein
import gensim

import pandas as pd
import seaborn as sns
import numpy as np

##### Carga de datos

In [3]:
with open (r'colores.html', 'r') as f:
    page = f.read()
    tree = html.fromstring(page)

In [9]:
elemento = tree.xpath ( '//div[@id = "ariketa1"]/div[@class = "wrapper"]/div[@class = "box amarillo"]/text()')
elemento

['Python']

In [12]:
texto_importante = tree.xpath('//p[@id = "muestra"]/text()')
texto_importante

[' Pyhthon, Python, Pythhon, Matlan, Matlas, Matalab, Maltab, Jaava, Jara, Lava, Java, Spar, Sppark, Spaar, Sapakr, Scala, Slaca, Salac, Scalal, Clojutre, Clourem, Clojurr, Cutre, Gorvo, Grovvoy, Grrovy,Pyhthon, Python, Pythhon, Matlan, Matlas, Matalab, Maltab, Jaava, Jara, Lava, Java, Spar, Sppark, Spaar, Sapakr, Scala, Slaca, Salac, Scalal, Clojutre, Clourem, Clojurr, Cutre, Gorvo, Grovvoy, Grrovy,Pyhthon, Python, Pythhon, Matlan, Matlas, Matalab, Maltab, Jaava, Jara, Lava, Java, Spar, Sppark, Spaar, Sapakr, Scala, Slaca, Salac, Scalal, Clojutre, Clourem, Clojurr, Cutre, Gorvo, Grovvoy, Grrovy, Grovy']

In [18]:
elemento_bien= []
for line in elemento:
    elemento_bien = ([i.lower() for i in word_tokenize(line)])

palabras = []
for line in texto_importante:
    palabras.append([w.lower() for w in word_tokenize(line)])


In [20]:
elemento_bien

['python']

In [19]:
palabras

[['pyhthon',
  ',',
  'python',
  ',',
  'pythhon',
  ',',
  'matlan',
  ',',
  'matlas',
  ',',
  'matalab',
  ',',
  'maltab',
  ',',
  'jaava',
  ',',
  'jara',
  ',',
  'lava',
  ',',
  'java',
  ',',
  'spar',
  ',',
  'sppark',
  ',',
  'spaar',
  ',',
  'sapakr',
  ',',
  'scala',
  ',',
  'slaca',
  ',',
  'salac',
  ',',
  'scalal',
  ',',
  'clojutre',
  ',',
  'clourem',
  ',',
  'clojurr',
  ',',
  'cutre',
  ',',
  'gorvo',
  ',',
  'grovvoy',
  ',',
  'grrovy',
  ',',
  'pyhthon',
  ',',
  'python',
  ',',
  'pythhon',
  ',',
  'matlan',
  ',',
  'matlas',
  ',',
  'matalab',
  ',',
  'maltab',
  ',',
  'jaava',
  ',',
  'jara',
  ',',
  'lava',
  ',',
  'java',
  ',',
  'spar',
  ',',
  'sppark',
  ',',
  'spaar',
  ',',
  'sapakr',
  ',',
  'scala',
  ',',
  'slaca',
  ',',
  'salac',
  ',',
  'scalal',
  ',',
  'clojutre',
  ',',
  'clourem',
  ',',
  'clojurr',
  ',',
  'cutre',
  ',',
  'gorvo',
  ',',
  'grovvoy',
  ',',
  'grrovy',
  ',',
  'pyhthon',
  ',',
  'pyt

In [28]:
veces = 0

for i in range(len(palabras[0])):
    if Levenshtein.distance(elemento_bien[0], palabras[0][i]) == 0:
        veces +=1
    
print('Basándonos en la distacia Levenshtein, la palabra Java aparece repetida', veces, 'veces')

Basándonos en la distacia Levenshtein, la palabra Java aparece repetida 3 veces


# ARIKETA 3 (4 puntos)

Vamos construir un sistema de recomendación de películas basandonos en la similitud del coseno. Para ello cargaremos el csv "movie_dataset.csv" un dataframe

In [31]:
df = pd.read_csv("movie_dataset.csv")
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

Una vez cargado el dataframe, vamos a realizar un simple preproceso, cambiando en las columnas ['keywords', 'cast', 'genres', 'director'] los valores NaN por ''.


In [35]:
df[['keywords', 'cast', 'genres', 'director']] = df[['keywords', 'cast', 'genres', 'director']].fillna('')

Entre todas estas características, las que nos interesan para encontrar la similitud para hacer la  recomendación son las siguientes: keywords, cast, genres y director. Para ello, vamos a crearnos una nueva columna df['combined_features'] concatenando los strings de las columnas keywords, cast, genres y director.

In [36]:
def combined_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']
df["combined_features"] = df.apply(combined_features, axis =1)

In [37]:
df["combined_features"]

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800    date love at first sight narration investigati...
4801     Daniel Henney Eliza Coupe Bill Paxton Alan Ru...
4802    obsession camcorder crush dream girl Drew Barr...
Name: combined_features, Length: 4803, dtype: object

La función anterior combinará todas nuestras features(palabras clave, elenco, géneros y director) de sus respectivas filas, y devolverá una fila con todas las características combinadas en un solo string.

Una vez creada esta nueva columna df["combined_features"], creamos la matriz de vectores y su matriz de similitud del coseno

In [45]:
vectors = CountVectorizer().fit_transform(df["combined_features"])
vectors = vectors.toarray()
vectors



array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
csim = cosine_similarity(vectors)
csim

array([[1.        , 0.10540926, 0.12038585, ..., 0.        , 0.        ,
        0.        ],
       [0.10540926, 1.        , 0.0761387 , ..., 0.03651484, 0.        ,
        0.        ],
       [0.12038585, 0.0761387 , 1.        , ..., 0.        , 0.11145564,
        0.        ],
       ...,
       [0.        , 0.03651484, 0.        , ..., 1.        , 0.        ,
        0.04264014],
       [0.        , 0.        , 0.11145564, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 0.        ,
        1.        ]])

In [49]:
len(csim)

4803

Si todo ha ido bien, la matriz de similitud debería tener esta pinta.

Ahora, debeis obtener del dataframe el index del título de la película escrapeado de la página https://dreguera.github.io. Acordaos de utilizar df.values para obtener el valor concreto.

In [67]:
titulo = tree.xpath('//div[@id = "ariketa2"]/div[@class = "box amarillo"]/text()')
titulo = titulo[0]
titulo

'Interstellar'

In [68]:
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [70]:
movie_index = get_index_from_title(titulo)
movie_index

95

Esta función la debeis utilizar para obtener el índice del nombre de esta película. El índice lo deberíais guardar en una variable llamada por ejemplo movie_index

In [72]:
print(titulo + ': ' + str(movie_index))

Interstellar: 95


Los indices os deberían dar algo así:

Ahora, vamos a buscar las películas similares en la matriz de similitud. Para ello vamos a utilizar la siguiente función que me devuelve los indices de las películas y el coeficiente de similitud.



Para ello usaremos el movie_index de la película que hemos escrapeado. El método enumerate() agregará un contador a la lista iterable cosine_sim y la devolverá en forma de lista de películas similares con el puntaje de similitud de cada índice.

In [76]:
similar_movies = list(enumerate(csim[movie_index])) 
similar_movies

#Donde CSIM es vuestra matriz de similitudes y movie_index el indice que habeis calculado en el paso anterior

[(0, 0.11111111111111113),
 (1, 0.03513641844631533),
 (2, 0.040128617695256406),
 (3, 0.264197974633739),
 (4, 0.11547005383792516),
 (5, 0.03849001794597506),
 (6, 0.0),
 (7, 0.10540925533894599),
 (8, 0.08206099398622181),
 (9, 0.07856742013183862),
 (10, 0.23094010767585035),
 (11, 0.03849001794597506),
 (12, 0.040128617695256406),
 (13, 0.03849001794597506),
 (14, 0.2545875386086578),
 (15, 0.07548513560963972),
 (16, 0.10369516947304253),
 (17, 0.041030496993110906),
 (18, 0.1405456737852613),
 (19, 0.041030496993110906),
 (20, 0.07856742013183862),
 (21, 0.03928371006591931),
 (22, 0.04415107856883479),
 (23, 0.04303314829119352),
 (24, 0.06804138174397717),
 (25, 0.04303314829119352),
 (26, 0.10910894511799618),
 (27, 0.11785113019775793),
 (28, 0.11785113019775793),
 (29, 0.041030496993110906),
 (30, 0.03774256780481986),
 (31, 0.11547005383792516),
 (32, 0.10910894511799618),
 (33, 0.10910894511799618),
 (34, 0.0),
 (35, 0.16798421022632323),
 (36, 0.15713484026367724),
 (37,

Ahora, vamos a ordenar la lista creada similar_movies de manera descendente en base a su coeficiente de similitud del coseno. Para ello utilizar la siguiente función que lo único que hace es ordenar descendentemente de manera rápida y fácil

In [79]:
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)
sorted_similar_movies

#Debería mostraros algo tal que asi la lista sorted_similar_movies

[(95, 1.0000000000000004),
 (123, 0.32732683535398854),
 (2375, 0.32102894156205125),
 (125, 0.31622776601683794),
 (3439, 0.3142696805273545),
 (270, 0.31426968052735443),
 (149, 0.30792014356780045),
 (634, 0.30194054243855895),
 (108, 0.29629629629629634),
 (3688, 0.2765204519281134),
 (1358, 0.27498597046143514),
 (43, 0.2694301256218254),
 (2433, 0.2694301256218254),
 (1446, 0.26419797463373906),
 (3, 0.264197974633739),
 (93, 0.2592592592592593),
 (813, 0.2592592592592593),
 (14, 0.2545875386086578),
 (673, 0.2545875386086578),
 (870, 0.2545875386086578),
 (3679, 0.25197631533948484),
 (257, 0.24077170617153845),
 (583, 0.24077170617153845),
 (2862, 0.24077170617153845),
 (1296, 0.2381448361039201),
 (577, 0.23570226039551584),
 (720, 0.23570226039551584),
 (2068, 0.23103442669455732),
 (10, 0.23094010767585035),
 (363, 0.23094010767585035),
 (3373, 0.22680460581325726),
 (212, 0.22645540682891915),
 (365, 0.22645540682891915),
 (922, 0.22222222222222224),
 (545, 0.21821789023599

Ahora lo que debéis hacer es mostrar el título de las 15 primeras películas. Para ello, primero debeis obtener el título del df del indice y luego printearlo. 

In [82]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

for movie in similar_movies[0:15]: #ACORDAOS QUE SOLO QUEREMOS LAS 15 PRIMERAS
    print(get_title_from_index(movie[0]))

Avatar
Pirates of the Caribbean: At World's End
Spectre
The Dark Knight Rises
John Carter
Spider-Man 3
Tangled
Avengers: Age of Ultron
Harry Potter and the Half-Blood Prince
Batman v Superman: Dawn of Justice
Superman Returns
Quantum of Solace
Pirates of the Caribbean: Dead Man's Chest
The Lone Ranger
Man of Steel


# ARIKETA 4 (2 puntos)

Dados los ficheros noticia1.txt y noticia2.txt mediante la librería gensim devolver por pantalla (SÓLO) el porcentaje de similitud entre los dos textos. (Nota: Se pude hacer uso de las librerías vistas en clase para limpieza de texto)